In [ ]:
#worsened score from 0.992 to 1.001
#improved SVM score from 1.382 to 1.139
#dfnn=pd.read_csv("../input/mergeneural512newparamsless/mergeNeuralNTPost99.csv")

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
pdf=pd.read_csv("../input/mergeneural512newparamsless/mergeNeuralNTPost99.csv")
pdf.shape


In [ ]:
pdf.describe()

In [ ]:
import copy

def alterProbabilities(odf, sqrt=False):
    df=copy.deepcopy(odf)
    for cindex in df.columns:
        df['adjSum']=0
        if cindex != 'object_id':
            df.loc[:,'not_' + str(cindex)]=1.0 - df.loc[:,cindex]
            zeroFilter=(df.loc[:,'not_' + str(cindex)]==0)
            df.loc[zeroFilter,'adj_' + str(cindex)]=999999
            df.loc[~zeroFilter,'adj_' + str(cindex)]=df.loc[~zeroFilter,cindex] / df.loc[~zeroFilter,'not_' + str(cindex)]
            if sqrt:
                df['adj_' + str(cindex)]=np.sqrt(df['adj_' + str(cindex)])
                
            #df.loc[:,'adjSum']+=df.loc[:,'adj_' + str(cindex)]
    
    for cindex in odf.columns:
        if cindex !='object_id':
            df.loc[:,'adjSum']+=df.loc[:,'adj_' + str(cindex)]
        

    for cindex in odf.columns:
        if cindex !='object_id':
            odf.loc[:,cindex]=df.loc[:,'adj_' + str(cindex)] / df.loc[:,'adjSum']
        
    return odf, df

s, os=alterProbabilities(pdf)
s.head()


In [ ]:
os.head()

In [ ]:
s.describe()

In [ ]:
os.describe()

In [ ]:
#from Scirpus discussion:

def GenUnknown(data):
    return ((((((data["mymedian"]) + (((data["mymean"]) / 2.0)))/2.0)) + (((((1.0) - (((data["mymax"]) * (((data["mymax"]) * (data["mymax"]))))))) / 2.0)))/2.0)

feats = ['class_6', 'class_15', 'class_16', 'class_42', 'class_52', 'class_53',
         'class_62', 'class_64', 'class_65', 'class_67', 'class_88', 'class_90',
         'class_92', 'class_95']

y = pd.DataFrame()
y['mymean'] = s[feats].mean(axis=1)
y['mymedian'] = s[feats].median(axis=1)
y['mymax'] = s[feats].max(axis=1)

s['class_99'] = GenUnknown(y)

In [ ]:
s.describe()

In [ ]:
s.to_csv('moreDecisiveNeural.csv', index=False)